In [33]:
import os
import sys

cwd = os.getcwd()
root_dir = os.path.dirname(os.path.dirname(cwd))
sys.path.append(root_dir)

print(root_dir)

c:\Users\david\Desktop\Main\03_Uni\WWI21DSA\02_Vorlesungen\06_Projektrealisierung\Projektrealisierung


In [34]:
import pandas as pd
import random

pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
pd.set_option("display.max_colwidth", None)

In [35]:
abcd = pd.read_csv("../../src/data/ABCD_tripfiles_preprocessed.csv")
mnop = pd.read_csv("../../src/data/MNOP_tripfiles_preprocessed.csv")
zyxw = pd.read_csv("../../src/data/ZYXW_tripfiles_preprocessed.csv")

C:\Users\david\AppData\Local\Temp\ipykernel_4328\946881633.py:1: DtypeWarning: Columns (12,13,14,15,16,17,18,19,20,21,22,24,25) have mixed types. Specify dtype option on import or set low_memory=False.
  abcd = pd.read_csv("../../src/data/ABCD_tripfiles_preprocessed.csv")
C:\Users\david\AppData\Local\Temp\ipykernel_4328\946881633.py:2: DtypeWarning: Columns (12,13,14,15,16,17,18,19,20,21,22,23,25,26) have mixed types. Specify dtype option on import or set low_memory=False.
  mnop = pd.read_csv("../../src/data/MNOP_tripfiles_preprocessed.csv")
C:\Users\david\AppData\Local\Temp\ipykernel_4328\946881633.py:3: DtypeWarning: Columns (12,13,14,15,16,17,18,19,20,21,22,24,25,26,27) have mixed types. Specify dtype option on import or set low_memory=False.
  zyxw = pd.read_csv("../../src/data/ZYXW_tripfiles_preprocessed.csv")


In [38]:
abcd_cols = set(abcd.columns)
mnop_cols = set(mnop.columns)
zyxw_cols = set(zyxw.columns)


abcd_cols.symmetric_difference(zyxw_cols)

{'DEADLOAD_MAC',
 'DEZYLOZY_MAC',
 'ESTIMATED_TRAFFIC_LOAD',
 'ESTIMATED_TRAFFIC_LOZY',
 'IDEAL_ADDITIONAL_LOAD_AFT',
 'IDEAL_ADDITIONAL_LOAD_FWD',
 'IDEAL_ZYDITIONAL_LOZY_AFT',
 'IDEAL_ZYDITIONAL_LOZY_FWD',
 'LOAD_TO_AFT',
 'LOAD_TO_FWD',
 'LOZY_TO_AFT',
 'LOZY_TO_FWD',
 'TOTAL_DEADLOAD_WI_index',
 'TOTAL_DEADLOAD_WI_weight',
 'TOTAL_DEZYLOZY_WI_index',
 'TOTAL_DEZYLOZY_WI_weight',
 'TOTAL_LOAD_WI',
 'TOTAL_LOZY_WI',
 'TOTAL_TRAFFIC_LOAD',
 'TOTAL_TRAFFIC_LOZY',
 'UNDERLOAD',
 'UNDERLOZY',
 'acarsActual',
 'dei'}

{'DEADLOAD_MAC',
 'DEZYLOZY_MAC',
 'ESTIMATED_TRAFFIC_LOAD',
 'ESTIMATED_TRAFFIC_LOZY',
 'IDEAL_ADDITIONAL_LOAD_AFT',
 'IDEAL_ADDITIONAL_LOAD_FWD',
 'IDEAL_ZYDITIONAL_LOZY_AFT',
 'IDEAL_ZYDITIONAL_LOZY_FWD',
 'LOAD_TO_AFT',
 'LOAD_TO_FWD',
 'LOZY_TO_AFT',
 'LOZY_TO_FWD',
 'TOTAL_DEADLOAD_WI_index',
 'TOTAL_DEADLOAD_WI_weight',
 'TOTAL_DEZYLOZY_WI_index',
 'TOTAL_DEZYLOZY_WI_weight',
 'TOTAL_LOAD_WI',
 'TOTAL_LOZY_WI',
 'TOTAL_TRAFFIC_LOAD',
 'TOTAL_TRAFFIC_LOZY',
 'UNDERLOAD',
 'UNDERLOZY',
 'acarsActual',
 'dei'}

In [36]:
print(
    abcd.columns.__len__(),
    mnop.columns.__len__(),
    zyxw.columns.__len__(),
)

76 77 78


In [14]:
meta = ['id', 'creation_time', 'airline_code', 'flight_number', 'flight_date',
       'departure_airport', 'user_name', 'action_name', 'header_line']
message = ['entry_details']

header_cols = ['header_category', 'header_id']

extracted_cols = ['scheduleState',
       'departureAirport', 'departureTime', 'arrivalAirport', 'arrivalTime',
       'aircraftRegistration', 'aircraftSubtype', 'aircraftVersion',
       'serviceType', 'nonOperational', 'flifoCancel', 'sequenceNumber',
       'bestTimes', 'mvtActual']

weight_cols = ['START_WI_weight', 'START_WI_index',
       'DO_WI_weight', 'DO_WI_index', 'PAX_WI_weight', 'PAX_WI_index',
       'TOTAL_DEADLOAD_WI_weight', 'TOTAL_DEADLOAD_WI_index', 'TOTAL_LOAD_WI',
       'TOTAL_TRAFFIC_LOAD', 'FUEL_INDEX', 'AZFW', 'ATOW', 'ALAW', 'ATXW',
       'AFT_LIMIT_ZFW', 'FWD_LIMIT_ZFW', 'AFT_LIMIT_TOW', 'FWD_LIMIT_TOW',
       'AFT_LIMIT_LAW', 'FWD_LIMIT_LAW', 'LIZFW', 'LITOW', 'LILAW',
       'MAC_AT_ZFW', 'MAC_AT_TOW', 'MAC_AT_LAW', 'DEADLOAD_MAC', 'UNDERLOAD',
       'LIMITING_WEIGHT', 'ALLOWED_TOW', 'ALLOWED_ZFW', 'ALLOWED_LAW',
       'ALLOWED_TXW', 'STABTO', 'OPTIMAL_TRIM', 'IDEAL_ADDITIONAL_LOAD_AFT',
       'IDEAL_ADDITIONAL_LOAD_FWD', 'TAIL_TIPPING_WI_weight',
       'TAIL_TIPPING_WI_index', 'TAIL_TIPPING_INDEX_EXCEEDED',
       'FWD_MOVABLE_PAX', 'AFT_MOVABLE_PAX', 'INDEX_OUT_OF_BALANCE',
       'LOAD_TO_AFT', 'LOAD_TO_FWD', 'ESTIMATED_TRAFFIC_LOAD', 'ESTIMATED_ZFW',
       'DELTA_ZFW', 'ZFW_TOLERANCE_EXCEEDED']


In [ ]:
temp = abcd.loc[
    (abcd["flight_number"] == 2312) &
    (abcd["action_name"] == "CalculateWeightAndTrimAction") &
    (abcd["header_category"] == "saved"),
    meta + header_cols + weight_cols
]

temp.sort_values("creation_time", inplace=True)

In [23]:
from IPython.display import display, HTML


def display_dataframe_fixed_size(df, width='1400px', height='600px'):
    html = f"""
    <div style="width: {width}; height: {height}; overflow: scroll; border: 1px solid black;">
        {df.to_html()}
    </div>
    """
    display(HTML(html))

# Display the DataFrame with fixed size and scrollbars
display_dataframe_fixed_size(temp.sort_values("creation_time"))

,id,creation_time,airline_code,flight_number,flight_date,departure_airport,user_name,action_name,header_line,header_category,header_id,START_WI_weight,START_WI_index,DO_WI_weight,DO_WI_index,PAX_WI_weight,PAX_WI_index,TOTAL_DEADLOAD_WI_weight,TOTAL_DEADLOAD_WI_index,TOTAL_LOAD_WI,TOTAL_TRAFFIC_LOAD,FUEL_INDEX,AZFW,ATOW,ALAW,ATXW,AFT_LIMIT_ZFW,FWD_LIMIT_ZFW,AFT_LIMIT_TOW,FWD_LIMIT_TOW,AFT_LIMIT_LAW,FWD_LIMIT_LAW,LIZFW,LITOW,LILAW,MAC_AT_ZFW,MAC_AT_TOW,MAC_AT_LAW,DEADLOAD_MAC,UNDERLOAD,LIMITING_WEIGHT,ALLOWED_TOW,ALLOWED_ZFW,ALLOWED_LAW,ALLOWED_TXW,STABTO,OPTIMAL_TRIM,IDEAL_ADDITIONAL_LOAD_AFT,IDEAL_ADDITIONAL_LOAD_FWD,TAIL_TIPPING_WI_weight,TAIL_TIPPING_WI_index,TAIL_TIPPING_INDEX_EXCEEDED,FWD_MOVABLE_PAX,AFT_MOVABLE_PAX,INDEX_OUT_OF_BALANCE,LOAD_TO_AFT,LOAD_TO_FWD,ESTIMATED_TRAFFIC_LOAD,ESTIMATED_ZFW,DELTA_ZFW,ZFW_TOLERANCE_EXCEEDED
180350,33813052,2024-04-30 13:30:53,AB,2312,10,PNQ,service-acco,CalculateWeightAndTrimAction,"2024-04-30 13:30:53,224 INFO [8cf9b0141315a4bb] [AB2312 /10 PNQ | service-acco | CalculateWeightAndTrimAction | | Thread-49237 (ActiveMQ-client-global-threads)] Saved:",saved,8cf9b0141315a4bb,44562.0,37.43,44976.0,37.61,0.0,0.00,44976.0,37.61,0.0,0.0,0.00,44976.0,44976.0,44976.0,44976.0,42.60,31.26,42.70,31.16,NaN,NaN,37.61,37.61,37.61,16.34,16.34,16.34,16.34,20976.0,NaN,65952.0,44976.0,44976.0,44976.0,NaN,30.70,-449.14,449.14,44976.00,37.61,NaN,NaN,NaN,NaN,NaN,NaN,0.00,44976.00,0.00,NaN
180349,33813053,2024-04-30 13:30:53,AB,2312,10,PNQ,service-acco,CalculateWeightAndTrimAction,"2024-04-30 13:30:53,225 INFO [8cf9b0141315a4bb] [AB2312 /10 PNQ | service-acco | CalculateWeightAndTrimAction | | Thread-49237 (ActiveMQ-client-global-threads)] Saved:",saved,8cf9b0141315a4bb,44562.0,37.43,NaN,NaN,0.0,0.00,NaN,NaN,0.0,0.0,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,NaN,NaN,NaN
180341,33813385,2024-04-30 13:30:55,AB,2312,5,PNQ,service-acco,CalculateWeightAndTrimAction,"2024-04-30 13:30:55,176 INFO [5161add68e60531b] [AB2312 /05 PNQ | service-acco | CalculateWeightAndTrimAction | | Thread-49237 (ActiveMQ-client-global-threads)] Saved:",saved,5161add68e60531b,44452.0,36.40,44866.0,36.58,0.0,0.00,44866.0,36.58,0.0,0.0,0.00,44866.0,44866.0,44866.0,44866.0,42.45,31.33,42.55,31.23,NaN,NaN,36.58,36.58,36.58,15.81,15.81,15.81,15.81,21086.0,NaN,65952.0,44866.0,44866.0,44866.0,NaN,30.55,-391.71,391.71,44866.00,36.58,NaN,NaN,NaN,NaN,NaN,NaN,0.00,44866.00,0.00,NaN
180342,33813386,2024-04-30 13:30:55,AB,2312,5,PNQ,service-acco,CalculateWeightAndTrimAction,"2024-04-30 13:30:55,176 INFO [5161add68e60531b] [AB2312 /05 PNQ | service-acco | CalculateWeightAndTrimAction | | Thread-49237 (ActiveMQ-client-global-threads)] Saved:",saved,5161add68e60531b,44452.0,36.40,NaN,NaN,0.0,0.00,NaN,NaN,0.0,0.0,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,NaN,NaN,NaN
180305,33815750,2024-04-30 13:31:14,AB,2312,14,PNQ,service-acco,CalculateWeightAndTrimAction,"2024-04-30 13:31:14,604 INFO [19f2554b7de0430f] [AB2312 /14 PNQ | service-acco | CalculateWeightAndTrimAction | | Thread-49241 (ActiveMQ-client-global-threads)] Saved:",saved,19f2554b7de0430f,44452.0,36.40,44866.0,36.58,0.0,0.00,44866.0,36.58,0.0,0.0,0.00,44866.0,44866.0,44866.0,44866.0,42.45,31.33,42.55,31.23,NaN,NaN,36.58,36.58,36.58,15.81,15.81,15.81,15.81,21086.0,NaN,65952.0,44866.0,44866.0,44866.0,NaN,30.55,-391.71,391.71,44866.00,36.58,NaN,NaN,NaN,NaN,NaN,NaN,0.00,44866.00,0.00,NaN
179764,33831434,2024-04-30 16:10:02,AB,2312,2,PNQ,service-acco,CalculateWeightAndTrimAction,"2024-04-30 16:10:02,874 INFO [a98bd5dfe31fa4cd] [AB2312 /02 PNQ | service-acco | CalculateWeightAndTrimAction | | Thread-49263 (ActiveMQ-client-global-threads)] Saved:",saved,a98bd5dfe31fa4cd,44537.0,37.95,NaN,NaN,0.0,0.00,NaN,NaN,0.0,0.0,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N

In [32]:
temp[temp["header_id"] == "d4b3b3b3-0b3b-4b3b-8b3b-3b3b3b3b3b3b3"]

SyntaxError: unmatched ']' (389530827.py, line 1)

In [ ]:
header_id -> Zur Kennzeichnung einzelner Flüge